In [1]:
import argparse
import os
import torch

from exp.exp_informer import Exp_Informer

In [2]:
from pyats.datastructures import AttrDict

In [3]:
args = AttrDict({"activation": 'gelu', "attn": 'prob', "batch_size": 32, 
        "c_out": 7, "checkpoints": './checkpoints/', "cols": None, 
        "d_ff": 2048, "d_layers": 1, "d_model": 320, "data": 'ETTh1', 
        "data_path": 'ETTh1.csv', "dec_in": 7, "des": 'Exp', "detail_freq": 'h', 
        "devices": '0,1,2,3', "distil": True, "do_predict": False, "dropout": 0.05, 
        "e_layers": 5, "embed": 'timeF', "enc_in": 7, "factor": 5, "features": 'M', "freq": 'h', 
        "gpu": 0, "inverse": False, "itr": 1, "label_len": 48, "learning_rate": 0.0001, "loss": 'mse', 
        "lradj": 'type1', "mix": True, "model": 'informer-moco', "n_heads": 8, "num_workers": 0, 
        "output_attention": False, "padding": 0, "patience": 3, "pred_len": 24, "root_path": './data/ETT/', 
        "s_layers": [3, 2, 1], "seq_len": 48, "target": 'OT', "train_epochs": 1, "use_amp": False, 
        "use_gpu": True, "use_multi_gpu": False, "loss_lambda": 0.0, "mask_rate": 0.2, "kernel_size": 3, "l2norm": True,
                "moco_average_pool": False, "data_aug": "cost", "mare": True, "time_feature_embed": True})

In [4]:
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

In [5]:
if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

In [6]:
data_parser = {
    'ETTh1':{'data':'ETTh1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTh2':{'data':'ETTh2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm1':{'data':'ETTm1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm2':{'data':'ETTm2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'WTH':{'data':'WTH.csv','T':'WetBulbCelsius','M':[12,12,12],'S':[1,1,1],'MS':[12,12,1]},
    'ECL':{'data':'ECL.csv','T':'MT_320','M':[321,321,321],'S':[1,1,1],'MS':[321,321,1]},
    'Solar':{'data':'solar_AL.csv','T':'POWER_136','M':[137,137,137],'S':[1,1,1],'MS':[137,137,1]},
}
if args.data in data_parser.keys():
    data_info = data_parser[args.data]
    args.data_path = data_info['data']
    args.target = data_info['T']
    args.enc_in, args.dec_in, args.c_out = data_info[args.features]

# args.s_layers = [int(s_l) for s_l in args.s_layers.replace(' ','').split(',')]
args.detail_freq = args.freq
args.freq = args.freq[-1:]
args.time_feature_embed = False
args.mare = False
print('Args in experiment:')
print(args)

Exp = Exp_Informer

Args in experiment:
AttrDict({'activation': 'gelu', 'attn': 'prob', 'batch_size': 32, 'c_out': 7, 'checkpoints': './checkpoints/', 'cols': None, 'd_ff': 2048, 'd_layers': 1, 'd_model': 320, 'data': 'ETTh1', 'data_path': 'ETTh1.csv', 'dec_in': 7, 'des': 'Exp', 'detail_freq': 'h', 'devices': '0,1,2,3', 'distil': True, 'do_predict': False, 'dropout': 0.05, 'e_layers': 5, 'embed': 'timeF', 'enc_in': 7, 'factor': 5, 'features': 'M', 'freq': 'h', 'gpu': 0, 'inverse': False, 'itr': 1, 'label_len': 48, 'learning_rate': 0.0001, 'loss': 'mse', 'lradj': 'type1', 'mix': True, 'model': 'informer-moco', 'n_heads': 8, 'num_workers': 0, 'output_attention': False, 'padding': 0, 'patience': 3, 'pred_len': 24, 'root_path': './data/ETT/', 's_layers': [3, 2, 1], 'seq_len': 48, 'target': 'OT', 'train_epochs': 1, 'use_amp': False, 'use_gpu': False, 'use_multi_gpu': False, 'loss_lambda': 0.0, 'mask_rate': 0.2, 'kernel_size': 3, 'l2norm': True, 'moco_average_pool': False, 'data_aug': 'cost', 'mare': False, 'ti

In [7]:
from models.dtcn_encoder import *
from models.tcn_encoder import *
from models.lstm_encoder import LSTM_Encoder
from models.informer_encoder import Informer_Encoder

In [8]:
encoder = Informer_Encoder(320, d_model=128, factor=2, d_ff=64, e_layer=5, distil=False)

In [9]:
pytorch_total_params = sum(p.numel() for p in encoder.parameters() if p.requires_grad)
print(pytorch_total_params)

457216


In [10]:
x = torch.rand([32, 128, 96])

In [11]:
for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                args.seq_len, args.label_len, args.pred_len,
                args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, 
                args.embed, args.distil, args.mix, args.des, ii)

    exp = Exp(args) # set experiments
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp.train(setting)

Use CPU
Use Data augmentation method: cost
l2norm True
Mask Rate: 0.5
[INFO] NOT Using Time Features.
[INFO] Number of parameters:  768647
TCN_MoCo(
  (encoder_q): TCNBase(
    (enc_embedding): Linear(in_features=7, out_features=128, bias=True)
    (encoder): Informer_Encoder(
      (encoder): Encoder(
        (attn_layers): ModuleList(
          (0): EncoderLayer(
            (attention): AttentionLayer(
              (inner_attention): ProbAttention(
                (dropout): Dropout(p=0.05, inplace=False)
              )
              (query_projection): Linear(in_features=128, out_features=128, bias=True)
              (key_projection): Linear(in_features=128, out_features=128, bias=True)
              (value_projection): Linear(in_features=128, out_features=128, bias=True)
              (out_projection): Linear(in_features=128, out_features=128, bias=True)
            )
            (conv1): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
            (conv2): Conv1d(64, 128, kernel

train 8569
val 2857
test 2857
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
	iters: 1, epoch: 1 | loss: 1.5543658
	speed: 0.4414s/iter; left time: 117.8537s
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch

torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([

torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([32, 320, 48])
torch.Size([

AttributeError: 'AttrDict' object has no attribute 'cos_lr'

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(exp.model)

In [ ]:
16265749 16528385

In [ ]:
#TCN
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [ ]:
x = torch.rand(4, 128, 72)

In [ ]:
tcn = TemporalConvNet(128, [512,512], dropout=0.1)

In [ ]:
tcn(x).shape

In [ ]:

lstm = nn.LSTM(128, 512, num_layers=3, batch_first=True, dropout=0.1)

In [ ]:
x = torch.rand(4, 72, 128)
out, (hn, cn) = lstm(x)

In [ ]:
decoder = nn.Linear(512, 7, bias=True)

In [ ]:
decoder(out).shape

In [ ]:
len(out)

In [20]:
#TCN
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.nn.utils import weight_norm
from models.embed import TimeFeatureEmbedding
import sys, math, random, copy
from typing import Union, Callable, Optional, List
from models.weight_norm import WeightNorm
from models.data_aug import Data_Aug

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = WeightNorm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation), ['weight'])
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = WeightNorm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation), ['weight'])
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.module.weight_v.data.normal_(0, 0.01)
        self.conv2.module.weight_v.data.normal_(0, 0.01)
        self.conv1.module.weight_g.data.normal_(0, 0.01)
        self.conv2.module.weight_g.data.normal_(0, 0.01)
        
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [21]:
from pytorch_receptive_field.torch_receptive_field import receptive_field
# from models.tcn_encoder import *

In [15]:
encoder = DilatedConvEncoder(64,  [64]*10 + [320], 3)



In [17]:
encoder

DilatedConvEncoder(
  (net): Sequential(
    (0): ConvBlock(
      (conv1): SamePadConv(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      )
      (conv2): SamePadConv(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      )
    )
    (1): ConvBlock(
      (conv1): SamePadConv(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
      )
      (conv2): SamePadConv(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
      )
    )
    (2): ConvBlock(
      (conv1): SamePadConv(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
      )
      (conv2): SamePadConv(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
      )
    )
    (3): ConvBlock(
      (conv1): SamePadConv(
        (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
      )
  

In [24]:
x = torch.rand([4, 64, 96])

In [29]:
receptive_field_dict = receptive_field(encoder.network[0], (64, 96))
# receptive_field_for_unit(receptive_field_dict, "2", (101))

AttributeError: 'TemporalBlock' object has no attribute 'TemporalBlock'

In [35]:
for name, module in encoder.named_modules():
    print(str(module.__class__).split(".")[-1].split("'")[0])
    if str(module.__class__).split(".")[-1].split("'")[0] == "Conv1d":
        break

TemporalConvNet
Sequential
TemporalBlock
WeightNorm
Conv1d


In [39]:
module.dilation

(1,)

In [18]:
class TemporalBlock(nn.Module):
    def __init__(self):
        super(TemporalBlock, self).__init__()
        self.conv = nn.Conv1d(3, 6, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv1d(6, 6, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.nettt = nn.Sequential(self.conv, self.conv2)
    def forward(self, x):
        
        return self.nettt(x)

class Net1D(nn.Module):
    def __init__(self):
        super(Net1D, self).__init__()
        self.net = TemporalBlock()
        self.bn = nn.BatchNorm1d(6)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)

    def forward(self, x):
        y = self.net(x)
        y = self.bn(y)
        y = self.relu(y)
        y = self.maxpool(y)
        return y

In [19]:
model = Net1D()
receptive_field_dict = receptive_field(model, (3, 16))

Conv1d(3, 6, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
in Conv1d
---------
Conv1d(6, 6, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
in Conv1d
---------
Conv1d(3, 6, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
in Conv1d
---------
Conv1d(6, 6, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
in Conv1d
---------
BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
in BatchNorm1d
---------
MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
in MaxPool1d
---------
6
Conv1d(3, 6, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
class_name Conv1d
Conv1d(3, 6, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
class_name Conv1d
Conv1d(6, 6, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
class_name Conv1d
Conv1d(6, 6, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
class_name Conv1d
BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
class

In [4]:
from collections import OrderedDict

xx =OrderedDict([('0', OrderedDict([('j', 1.0), ('r', 1.0), ('start', 0.5), ('conv_stage', True), ('output_shape', [-1, 64, 96])])), ('1', OrderedDict([('input_shape', [-1, 64, 96]), ('output_shape', [-1, 64, 98])])), ('2', OrderedDict())])

In [10]:
xx["2"]

OrderedDict()